In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd 
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [26]:
proteins_cleaned = pd.read_csv("./Data/joined_tables.csv")

Because previous methods based on simple sequence characteristics were insufficient, I decided to analyse the actual information from the sequence.

An LSTM model was initially selected, this model was my first choice because it seemed suitable, as this type of network can identify both shorter motifs and their relationships within the sequence.  

PyTorch was used to define the neural network, with various hyperparameters tested to achieve the best possible results.

# LSTM

First, the sequences were adjusted to have equal lengths by trimming longer sequences and applying padding to shorter ones. 

Different sequence lengths were tested, starting with 50 amino acids as the maximum due to computational limitations. Additionally, sequences of 20, 10, and 5 amino acids were testes. Surprisingly, the best results were achieved using a combination of the 5 initial and 5 amino acids from the end of a sequence 

In [27]:
def trimming_padding(sequence, leng, direction):

    if direction == "beg":
        new_seq = sequence[:leng]
        
        if len(new_seq) <= leng:
            fill = leng-len(new_seq)
            fill_seq = fill*"J"
            new_seq = new_seq + fill_seq
    else:
        new_seq = sequence[-leng:]
        
        if len(new_seq) <= leng:
            fill = leng-len(new_seq)
            fill_seq = fill*"J"
            new_seq = fill_seq + new_seq

    return new_seq

In [28]:
proteins_cleaned["trim_seq"] = proteins_cleaned["sequence"].apply(lambda x: trimming_padding(x,5, "beg"))
proteins_cleaned["trim_seq_end"] = proteins_cleaned["sequence"].apply(lambda x: trimming_padding(x,5,"end"))

Next, the encoding for the sequences had to be determined. To avoid adding too much dimensionality, encoding the letters of the alphabet in numerical order was considered. However, as it could introduce false relationships between elements within a sequence, one-hot encoding was ultimately chosen. While this method increased dimensionality, it still resulted in fewer dimensions than most embeddings.

In [29]:
aminoacids = ['A', 'C','D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R','S', 'T', 'U', 'V', 'W', 'Y', 'X', 'B', 'Z']
encoding = [np.zeros(len(aminoacids)) for a in aminoacids]

for idx, enc in enumerate(encoding):
    enc[idx] = 1

In [30]:
aminoacids_dict = dict(zip(aminoacids,encoding))
aminoacids_dict.update({"J":np.zeros(len(aminoacids))})

In [31]:
aminoacids_dict["A"]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.])

In [32]:
def map_seq(dict, sequence):

    array = np.zeros((len(sequence), 25))

    for idx, a in enumerate(sequence):
        array[idx] = dict[a]

    return array

Beginning and end sequences were combined together in one sequence

In [33]:
proteins_cleaned["seq_encoded"] = proteins_cleaned["trim_seq"].apply(lambda x: map_seq(aminoacids_dict,x))
proteins_cleaned["seq_encoded_end"] = proteins_cleaned["trim_seq_end"].apply(lambda x: map_seq(aminoacids_dict,x))

In [34]:
start_seq = np.array(proteins_cleaned["seq_encoded"].to_list())
end_seq = np.array(proteins_cleaned["seq_encoded_end"].to_list())

In [35]:
start_seq.shape

(117513, 5, 25)

In [36]:
proteins_dict = dict(zip(list(proteins_cleaned["classification"].value_counts().index), range(len(proteins_cleaned["classification"].value_counts().index.tolist()))))

In [37]:
seq = np.concatenate((start_seq, end_seq), axis=1)

In [38]:
X = seq
y = np.array(proteins_cleaned["classification"].map(proteins_dict))

In [ ]:
# np.save('./Data/x_lstm.npy', np.array(seq))
# np.save('./Data/y_lstm.npy', np.array(proteins_cleaned["classification"].map(proteins_dict)))

In [142]:
train_size = int(len(proteins_cleaned)*0.6)
test_size = int(len(proteins_cleaned)*0.2)

X_train = X[:train_size]
X_test = X[train_size:(train_size+test_size)]
X_val = X[(train_size+test_size):]

y_train = y[:train_size]
y_test = y[train_size:(train_size+test_size)]
y_val = y[(train_size+test_size):]

In [ ]:
freq_dict = dict(zip(pd.DataFrame(y_train)[0].value_counts().index,
                     pd.DataFrame(y_train)[0].value_counts().values.tolist()))

dict_weights = dict()

weights_array = np.zeros(len(freq_dict))

for key, value in freq_dict.items():
    weight = 1-(value/len(y_train))
    weights_array[key] = weight

In [ ]:
# np.save('./Data/weights.npy', weights_array)

In [144]:
X_train.shape

(70507, 10, 25)

In [145]:
X_train_tensor_lstm = torch.tensor(X_train,dtype=torch.float32)
y_train_tensor_lstm = torch.tensor(np.array(y_train), dtype=torch.long)

X_test_tensor_lstm = torch.tensor(X_test,dtype=torch.float32)
y_test_tensor_lstm = torch.tensor(y_test, dtype=torch.long)

X_val_tensor_lstm = torch.tensor(X_val,dtype=torch.float32)
y_val_tensor_lstm = torch.tensor(y_val, dtype=torch.long)

Bidirectional as well as unidirectional networks were tested, with multiple values of numbers of layers and hidden size. 

The best results were achieved with bidirectional network, with 3 layers and hidden size 128.

In [146]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        
        self.fc = nn.Linear(2*hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(2*self.num_layers, x.size(0), self.hidden_size)
        c0 = torch.zeros(2*self.num_layers, x.size(0), self.hidden_size)
        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [ ]:
# UNDIRECTIONAL MODEL

# class LSTMClassifier(nn.Module):
#     def __init__(self, input_size, hidden_size, num_layers, output_size):
#         super(LSTMClassifier, self).__init__()
#         self.hidden_size = hidden_size
#         self.num_layers = num_layers
#         self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
#         self.fc = nn.Linear(hidden_size, output_size)

#     def forward(self, x):
#         h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
#         c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
#         out, (hn, cn) = self.lstm(x, (h0, c0))
#         out = self.fc(out[:, -1, :])
#         return out

In [116]:
torch.any(torch.isnan(X_train_tensor_lstm))

tensor(False)

To prevent overfitting during the training loop, a simple version of early stopping was implemented. According to the definition: training was stopped when the loss on the training set began to increase while the loss on the validation set continued to decrease. Additionally, the best model was saved to be used for prediction in case it wasn't the one produced by the final epochs.

Both SGD and Adam optimizers were tested, with learning rates ranging from 0.001 to 0.1. The learning rate was increased to accelerate the learning process and decreased when fluctuations in the loss were observed during training.

In [147]:
input_size = 25
hidden_size = 128
num_layers = 3
output_size = 33

model_lstm = LSTMClassifier(input_size, hidden_size, num_layers, output_size)
criterion = nn.CrossEntropyLoss(weight=torch.tensor(weights_array,dtype=torch.float32))
optimizer = optim.Adam(model_lstm.parameters(), lr=0.01)

In [ ]:
n_epochs = 15
batch_size = 128

test_loss_array = []
patience = 3
best_result = np.inf

for epoch in range(n_epochs):

    model_lstm.train()
    batches = len(X_train_tensor_lstm) // batch_size
    for batch in range(batches):
        
        optimizer.zero_grad()
        i = batch * batch_size

        X_batch = X_train_tensor_lstm[i:i+batch_size]
        y_batch = y_train_tensor_lstm[i:i+batch_size]

        y_pred = model_lstm(X_batch)

        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()

    model_lstm.eval()
    batches = len(X_test_tensor_lstm) // batch_size
    test_loss = 0
    correct = 0
    with torch.no_grad():  

        for batch in range(batches):
            i = batch * batch_size

            X_batch = X_test_tensor_lstm[i:i+batch_size]
            y_batch = y_test_tensor_lstm[i:i+batch_size]

            output = model_lstm(X_batch)
            test_loss += criterion(output, y_batch).item()

    test_loss /= len(y_test_tensor_lstm)
    test_loss_array.append(test_loss)

    if test_loss < best_result:
        torch.save(model_lstm.state_dict(), "./lstm.pth")

    print(f"Epoch: {epoch}, Train loss: {loss}, Test loss: {test_loss}")

    if len(test_loss_array)>patience+1:
        if not (any(x > test_loss_array[-1] for x in test_loss_array[len(test_loss_array)-patience-1:-1])):
            break

Epoch: 0, Train loss: 2.495518684387207, Test loss: 0.019123774053248818
Epoch: 1, Train loss: 2.2585361003875732, Test loss: 0.016980705251288752
Epoch: 2, Train loss: 2.141655921936035, Test loss: 0.015828826946660757
Epoch: 3, Train loss: 1.9230124950408936, Test loss: 0.01518039873392772
Epoch: 4, Train loss: 1.6712727546691895, Test loss: 0.014659228284108813
Epoch: 5, Train loss: 1.7239651679992676, Test loss: 0.014560452020682636
Epoch: 6, Train loss: 1.5267056226730347, Test loss: 0.014588198279555367
Epoch: 7, Train loss: 1.439099907875061, Test loss: 0.014588282484874737
Epoch: 8, Train loss: 1.3084297180175781, Test loss: 0.01471036921135061


Best model was loaded from .pth file and used or a prediction on validation set

In [149]:
model_lstm.load_state_dict(torch.load("./lstm.pth"))

model_lstm.eval()
with torch.no_grad():
    predictions = model_lstm(X_val_tensor_lstm)
    predicted_labels = torch.argmax(predictions, dim=1)
    print("Predicted Labels:", predicted_labels)

Predicted Labels: tensor([ 2,  1,  3,  ..., 18,  1,  3])


In [150]:
results = pd.DataFrame({"pred":predicted_labels,"true":y_val})

In [151]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

The results still could be improved - i think the biggest change would be to use full sequences, but it is also possible that defining a function from the pure sequence is just way more complicated.


The results could still be improved. The most significant change would likely come from using full sequences. It is also possible that the sequence itself simply does not provide enough information and is not highly correlated with protein function.

In [152]:
y_test = results["true"]
y_pred = results["pred"]

print(f"accuracy: {accuracy_score(y_test, y_pred)}")
print(f"F1: {f1_score(y_test, y_pred, average='weighted')}")
print(f"precision: {precision_score(y_test, y_pred, average='weighted')}")
print(f"recall: {recall_score(y_test, y_pred, average='weighted')}")

accuracy: 0.5522038801906058
F1: 0.5501330616690168
precision: 0.5582311093258449
recall: 0.5522038801906058
